# 3. 應用 LoRA 進行模型微調

在本筆記本中，我們將進入最核心的環節：使用 `peft` 函式庫將 LoRA (Low-Rank Adaptation) 應用到我們已經載入的量化模型上，並設定訓練流程來進行微調。

## 步驟 1: 匯入必要的函式庫

首先，我們匯入 `peft` 函式庫中用於設定 LoRA 的 `LoraConfig` 和 `get_peft_model` 函式，以及 `transformers` 中用於設定訓練參數的 `TrainingArguments` 和執行訓練的 `Trainer`。


In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer
import torch

# 重新執行上一筆記本的程式碼，確保模型和 Tokenizer 已載入
# 在實際操作中，您也可以將模型和 Tokenizer 儲存後再載入，或使用其他方式在筆記本間共享變數
%run ./02-load-model-and-dataset.ipynb


## 步驟 2: 設定 LoRA (LoraConfig)

為了將 LoRA 應用到我們的模型上，我們需要建立一個 `LoraConfig` 物件。這個物件會告訴 `peft` 函式庫如何設定 LoRA adapter。

關鍵參數說明：
- `r`: LoRA 的秩 (rank)。這是一個關鍵超參數，決定了低秩矩陣的大小。較小的 `r` 意味著更少的參數和更快的訓練，但可能會犧牲一些性能。常見的設定值為 8, 16, 32, 64。
- `lora_alpha`: LoRA 的縮放因子，可以理解為學習率的縮放。公式為 `alpha / r`。通常設定為 `r` 的兩倍。
- `target_modules`: 指定要應用 LoRA 的模組名稱。對於 Transformer 模型，通常是注意力機制中的查詢 (query) 和值 (value) 投影層，即 `"q_proj"` 和 `"v_proj"`。
- `lora_dropout`: 在 LoRA 層中使用的 dropout 比率。
- `bias`: 設定 bias 是否可訓練。`"none"` 表示所有 bias 都不訓練。
- `task_type`: 指定任務類型。對於語言模型，我們設定為 `"CAUSAL_LM"`。

設定好 `LoraConfig` 後，我們使用 `get_peft_model` 函式將其應用到我們的基礎模型上。該函式會凍結所有原始模型參數，並在指定模組上加上 LoRA adapter。


In [ ]:
# 建立 LoraConfig
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 將 LoRA 應用到模型上
peft_model = get_peft_model(model, lora_config)

# 打印出模型中可訓練的參數
peft_model.print_trainable_parameters()


## 步驟 3: 設定訓練參數與啟動訓練

現在我們的模型已經準備好進行微調，最後一步是設定訓練流程。我們使用 Hugging Face `Trainer` 來處理大部分的訓練細節。

首先，建立一個 `TrainingArguments` 物件來定義訓練的各項參數：
- `output_dir`: 訓練過程中模型 checkpoint 和日誌的儲存目錄。
- `per_device_train_batch_size`: 每個 GPU 上的訓練批次大小。
- `gradient_accumulation_steps`: 梯度累積步數，用於在記憶體不足時模擬更大的批次大小。
- `learning_rate`: 學習率。
- `num_train_epochs`: 訓練的總輪數。
- `logging_steps`: 每隔多少步記錄一次訓練日誌。
- `fp16`: 啟用混合精度訓練以加速並節省記憶體。

接著，我們實例化一個 `Trainer`，並傳入模型、訓練參數、資料集等。最後，呼叫 `trainer.train()` 即可開始微調！

**注意**: 即使是 PEFT，微調 7B 模型仍然需要一些時間。在 Colab 的 T4 GPU 上，這個步驟大約需要 15-20 分鐘。


In [ ]:
# 設定訓練參數
training_args = TrainingArguments(
    output_dir="./lora-llama2-7b-chat",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    save_total_limit=2,
    overwrite_output_dir=True,
)

# 建立 Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    # 我們需要提供一個 data_collator 來將資料整理成批次
    # 對於語言模型，我們通常使用 DataCollatorForLanguageModeling
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# 關閉 cache 以解決一個已知的梯度檢查點問題
peft_model.config.use_cache = False

# 開始訓練
trainer.train()

print("
訓練完成！")


模型微調已經完成！在下一個筆記本 `04-inference-and-evaluation.ipynb` 中，我們將學習如何使用這個微調後的模型進行推理。
